# LMM predictions and prediction intervals

Below we will fit a linear mixed model using the Ruby gem [mixed\_models](https://github.com/agisga/mixed_models) and demonstrate the available prediction methods.

## Data and linear mixed model

We use the same data and model formulation as in several previous examples, where we have looked at various parameter estimates ([1](http://nbviewer.ipython.org/github/agisga/mixed_models/blob/master/notebooks/LMM_model_fitting.ipynb)) and demostrated many types hypotheses tests as well as confidence intervals ([2](http://nbviewer.ipython.org/github/agisga/mixed_models/blob/master/notebooks/LMM_tests_and_intervals.ipynb)).

The data set, which is simulated, contains two numeric variables *Age* and *Aggression*, and two categorical variables *Location* and *Species*. These data are available for 100 (human and alien) individuals.

We model the *Aggression* level of an individual of *Species* $spcs$ who is at the *Location* $lctn$ as:

$$Aggression = \beta_{0} + \gamma_{spcs} + Age \cdot \beta_{1} + b_{lctn,0} + Age \cdot b_{lctn,1} + \epsilon,$$

where $\epsilon$ is a random residual, and the random vector $(b_{lctn,0}, b_{lctn,1})^T$ follows a multivariate normal distribution (the same distribution but different realizations of the random vector for each *Location*).

We fit this model in `mixed_models` using a syntax familiar from the `R` package `lme4`.

In [1]:
require 'mixed_models'

alien_species = Daru::DataFrame.from_csv '../examples/data/alien_species.csv'
# mixed_models expects that all variable names in the data frame are ruby Symbols:
alien_species.vectors = Daru::Index.new(alien_species.vectors.map { |v| v.to_sym })

model_fit = LMM.from_formula(formula: "Aggression ~ Age + Species + (Age | Location)", 
                             data: alien_species)
model_fit.fix_ef_summary

#<Daru::DataFrame:47316264430760 @name = 2b874891-0fa8-4cca-9dc1-44a797bac352 @size = 5>
                 coef         sd    z_score WaldZ_p_va 
 intercept 1016.28672 60.1972749 16.8826034        0.0 
       Age -0.0653161 0.08988486 -0.7266646 0.46743141 
Species_lv -499.69369 0.26825234 -1862.7747        0.0 
Species_lv -899.56932 0.28144708 -3196.2289        0.0 
Species_lv -199.58895 0.27578357 -723.71589        0.0

## Predictions and prediction intervals

Often, the objective of a statistical model is the prediction of future observations based on new data input.

We consider the following new data set containing age, geographic location and species for ten individuals.

In [2]:
newdata = Daru::DataFrame.from_csv '../examples/data/alien_species_newdata.csv'
newdata.vectors = Daru::Index.new(newdata.vectors.map { |v| v.to_sym })
newdata

#<Daru::DataFrame:47316263806300 @name = c92169c8-786d-4326-b9bd-0d0c197a2f88 @size = 10>
                  Age    Species   Location 
         0        209      Dalek  OodSphere 
         1         90        Ood      Earth 
         2        173        Ood     Asylum 
         3        153      Human     Asylum 
         4        255 WeepingAng  OodSphere 
         5        256 WeepingAng     Asylum 
         6         37      Dalek      Earth 
         7        146 WeepingAng      Earth 
         8        127 WeepingAng     Asylum 
         9         41        Ood     Asylum

#### Point estimates

Based on the fitted linear mixed model we can predict the aggression levels for the inidividuals, where we can specify whether the random effects estimates should be included in the calculations or not.

In [3]:
puts "Predictions of aggression levels on a new data set:"
pred =  model_fit.predict(newdata: newdata, with_ran_ef: true)

Predictions of aggression levels on a new data set:


[1070.9125752531208, 182.45206492790737, -17.06446875476354, 384.7881586199103, 876.1240725686446, 674.7113391148862, 1092.6985606350866, 871.1508855262363, 687.4629975728096, -4.016260100144294]

Now we can add the computed predictions to the data set, in order to see better which of the individuals are likely to be particularly dangerous.

In [4]:
newdata = Daru::DataFrame.from_csv '../examples/data/alien_species_newdata.csv'
newdata.vectors = Daru::Index.new(newdata.vectors.map { |v| v.to_sym })
newdata[:Predicted_Agression] = pred
newdata

#<Daru::DataFrame:47316262633840 @name = e668eb36-b2da-4270-b771-4c8ff3a482f0 @size = 10>
                  Age    Species   Location Predicted_ 
         0        209      Dalek  OodSphere 1070.91257 
         1         90        Ood      Earth 182.452064 
         2        173        Ood     Asylum -17.064468 
         3        153      Human     Asylum 384.788158 
         4        255 WeepingAng  OodSphere 876.124072 
         5        256 WeepingAng     Asylum 674.711339 
         6         37      Dalek      Earth 1092.69856 
         7        146 WeepingAng      Earth 871.150885 
         8        127 WeepingAng     Asylum 687.462997 
         9         41        Ood     Asylum -4.0162601

#### Interval estimates

Since the estimated fixed and random effects coefficients most likely are not exactly the true values, we probably should look at interval estimates of the predictions, rather than the point estimates computed above.

Two types of such interval estimates are currently available in `LMM`. On the one hand, a *confidence interval* is an interval estimate of the mean value of the response for given covariates (i.e. a population parameter); on the other hand, a *prediction interval* is an interval estimate of a future observation (for further explanation of this distinction see for example <https://stat.ethz.ch/education/semesters/ss2010/seminar/06_Handout.pdf>).

In [5]:
puts "88% confidence intervals for the predictions:"
ci = model_fit.predict_with_intervals(newdata: newdata, level: 0.88, type: :confidence)
Daru::DataFrame.new(ci, order: [:pred, :lower88, :upper88])

88% confidence intervals for the predictions:


#<Daru::DataFrame:47316259596660 @name = 8d245d6c-289f-4825-b646-12c920e2015d @size = 10>
                 pred    lower88    upper88 
         0 1002.63564 906.275473 1098.99581 
         1 110.838945 17.1539311 204.523959 
         2 105.417704 10.1646879 200.670721 
         3 506.599653 411.851919 601.347388 
         4 800.042143 701.909117 898.175169 
         5 799.976827 701.800945 898.152709 
         6 1013.87002 920.443931 1107.29611 
         7 807.161604 712.571759 901.751449 
         8 808.402611 714.191640 902.613582 
         9 114.039437 20.6140348 207.464839

In [6]:
puts "88% prediction intervals for the predictions:"
pi = model_fit.predict_with_intervals(newdata: newdata, level: 0.88, type: :prediction)
Daru::DataFrame.new(pi, order: [:pred, :lower88, :upper88])

88% prediction intervals for the predictions:


#<Daru::DataFrame:47316258683700 @name = ef48b9a6-5554-4203-b935-bbb4eee42fe1 @size = 10>
                 pred    lower88    upper88 
         0 1002.63564 809.910050 1195.36123 
         1 110.838945 -76.536158 298.214049 
         2 105.417704 -85.093528 295.928938 
         3 506.599653 317.098899 696.100408 
         4 800.042143 603.771398 996.312888 
         5 799.976827 603.620377 996.333277 
         6 1013.87002 827.012723 1200.72732 
         7 807.161604 617.976730 996.346478 
         8 808.402611 619.975479 996.829743 
         9 114.039437 -72.816144 300.895018

**Remark**: You might notice that `#predict` with `with_ran_ef: true` produces some values outside of the confidence intervals, because the confidence intervals are computed from `#predict` with `with_ran_ef: false`.
However, `#predict` with `with_ran_ef: false` should always give values which lie in the center of the confidence or prediction intervals.